# PYSPARK CASE STUDY
# PALLAV GUPTA
# 46022374

## CONNECTING JUPYTER TO SPARK

In [3]:
import findspark
findspark.init()
findspark.find()

'c:\\spark'

## IMPORT NECESSARY LIBRARIES

In [4]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import udf
from pyspark.sql.functions import date_format
from pyspark.sql.functions import min,max,sum
from pyspark.sql.functions import weekofyear
from pyspark.sql.functions import isnan, when, count, col
import pyspark.sql.types as types
import math
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## IMPORTING DATA

In [3]:
uber_data1 = pd.read_csv('uber_data_nyc/part-00000-65909711-74cb-4da9-acbd-3c279273213e.csv.gz')
uber_data2 = pd.read_csv('uber_data_nyc/part-00001-65909711-74cb-4da9-acbd-3c279273213e.csv.gz')
uber_data3 = pd.read_csv('uber_data_nyc/part-00002-65909711-74cb-4da9-acbd-3c279273213e.csv.gz')
uber_data4 = pd.read_csv('uber_data_nyc/part-00003-65909711-74cb-4da9-acbd-3c279273213e.csv.gz')

## APPENDING DATA 

In [4]:
uber_data = uber_data4.append(uber_data3.append(uber_data2.append(uber_data1)))

In [5]:
uber_data.set_index(uber_data.index,inplace=True)

In [6]:
uber_data.head()

,VendorID,tpep_pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount
0,2,2016-06-09T21:06:36.000-07:00,-73.983360,40.760937,-73.977463,40.753979,7.30
1,2,2016-06-09T21:06:36.000-07:00,-73.987106,40.733173,-73.985909,40.766445,17.76
2,1,2016-06-09T21:06:37.000-07:00,-73.984276,40.775009,-73.968498,40.766430,9.30
3,2,2016-06-09T21:06:37.000-07:00,-73.976334,40.785851,-73.971733,40.791969,4.80
4,1,2016-06-09T21:06:38.000-07:00,-73.988029,40.743835,-73.990173,40.731773,8.75


In [7]:
len(uber_data)

3299999

In [8]:
uber_data.describe()

,VendorID,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount
count,3.299999e+06,3.299999e+06,3.299999e+06,3.299999e+06,3.299999e+06,3.299999e+06
mean,1.527929e+00,-7.305836e+01,4.024679e+01,-7.312868e+01,4.028653e+01,1.664624e+01
std,4.992194e-01,8.175607e+00,4.503776e+00,7.858150e+00,4.328784e+00,1.272999e+02
min,1.000000e+00,-1.151684e+02,0.000000e+00,-1.151754e+02,0.000000e+00,-1.953000e+02
25%,1.000000e+00,-7.399184e+01,4.073531e+01,-7.399123e+01,4.073413e+01,8.760000e+00
50%,2.000000e+00,-7.398138e+01,4.075307e+01,-7.397925e+01,4.075380e+01,1.230000e+01
75%,2.000000e+00,-7.396577e+01,4.076868e+01,-7.396158e+01,4.076994e+01,1.836000e+01
max,2.000000e+00,0.000000e+00,5.109830e+01,1.062469e+02,5.031214e+01,1.874423e+05


## CREATE INDEX

In [9]:
uber_data['index']=np.arange(len(uber_data))

In [10]:
uber_data.set_index('index')

,VendorID,tpep_pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount
index,,,,,,,
0,2,2016-06-09T21:06:36.000-07:00,-73.983360,40.760937,-73.977463,40.753979,7.30
1,2,2016-06-09T21:06:36.000-07:00,-73.987106,40.733173,-73.985909,40.766445,17.76
2,1,2016-06-09T21:06:37.000-07:00,-73.984276,40.775009,-73.968498,40.766430,9.30
3,2,2016-06-09T21:06:37.000-07:00,-73.976334,40.785851,-73.971733,40.791969,4.80
4,1,2016-06-09T21:06:38.000-07:00,-73.988029,40.743835,-73.990173,40.731773,8.75
5,2,2016-06-09T21:06:38.000-07:00,-74.016121,40.710899,-74.009789,40.715809,8.30
6,2,2016-06-09T21:06:39.000-07:00,-73.982590,40.777409,-73.966583,40.807934,14.30
7,1,2016-06-09T21:06:40.000-07:00,-73.970306,40.756084,-73.995689,40.764523,15.30
8,2,2016-06-09T21:06:40.000-07:00,-73.984703,40.754051,-74.002098,40.734707,12.96


## IMPORT DATA INO SPARK

In [11]:
sc = SparkContext() 
sqlContext = pyspark.SQLContext(sc)
spark = pyspark.sql.SparkSession(sc) 

In [12]:
DataFrame0 = spark.read.csv('uber_data_nyc/part-00000-65909711-74cb-4da9-acbd-3c279273213e.csv.gz'
                    ,header='true',inferSchema='true')
DataFrame1 = spark.read.csv('uber_data_nyc/part-00001-65909711-74cb-4da9-acbd-3c279273213e.csv.gz'
                    ,header='true',inferSchema='true')
DataFrame2 = spark.read.csv('uber_data_nyc/part-00002-65909711-74cb-4da9-acbd-3c279273213e.csv.gz'
                    ,header='true',inferSchema='true')
DataFrame3 = spark.read.csv('uber_data_nyc/part-00003-65909711-74cb-4da9-acbd-3c279273213e.csv.gz'
                    ,header='true',inferSchema='true')

In [13]:
DataFrame = DataFrame0.union(DataFrame1.union(DataFrame2.union(DataFrame3)))

In [14]:
DataFrame.count()

3299999

## 1. Use the Haversine formula to calculate the distance from the latitudes of the pickup and drop points. Include the formula in a Python package and create the necessary user defined functions in Spark. Store the data as a csv file.

In [15]:
def haversine(lon1,lat1,lon2,lat2): 
    dLon = (lon2-lon1)*math.pi/180.0
    dLat = (lat2-lat1)*math.pi/180.0

    lat1 = (lat1)*math.pi/180.0
    lat2 = (lat2)*math.pi/180.0
  
    a = (pow(math.sin(dLat/2),2)+pow(math.sin(dLon/2),2)*math.cos(lat1)*math.cos(lat2)); 
    rad = 6371
    c = 2*math.asin(math.sqrt(a)) 
    return (rad*c*0.621371)

In [16]:
haversine_udf = udf(haversine,types.DoubleType())
uber=DataFrame.withColumn("distance_miles",haversine_udf("pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"))

In [17]:
uber.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- distance_miles: double (nullable = true)



In [18]:
uber.show()

+--------+--------------------+------------------+------------------+------------------+------------------+------------+-------------------+
|VendorID|tpep_pickup_datetime|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|total_amount|     distance_miles|
+--------+--------------------+------------------+------------------+------------------+------------------+------------+-------------------+
|       2| 2016-06-10 09:36:36|-73.98171997070312| 40.73666763305664|-73.98163604736328| 40.67024230957031|        27.3|  4.589547091999834|
|       2| 2016-06-10 09:36:36|-73.99520111083984|  40.7394905090332|-73.99320220947266| 40.76264190673828|       14.16| 1.6030245428829144|
|       1| 2016-06-10 09:36:37|-73.97792053222656|40.750091552734375|-73.98723602294923|40.749359130859375|       16.63|0.49021864625329686|
|       2| 2016-06-10 09:36:37|-73.99358367919923| 40.72459030151367|-74.00804901123048| 40.74034118652344|       13.56| 1.3258730658656308|
|       2| 20

In [19]:
g=uber.select('VendorID','pickup_longitude','pickup_latitude',
                   'dropoff_longitude','dropoff_latitude','total_amount','distance_miles')
g.select([count(when(isnan(c), c)).alias(c) for c in g.columns]).show()

+--------+----------------+---------------+-----------------+----------------+------------+--------------+
|VendorID|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|total_amount|distance_miles|
+--------+----------------+---------------+-----------------+----------------+------------+--------------+
|       0|               0|              0|                0|               0|           0|             0|
+--------+----------------+---------------+-----------------+----------------+------------+--------------+



In [20]:
uber.write.csv('/Users/Sanjay Gupta/Desktop/NewUberData')

In [21]:
uber.createOrReplaceTempView('uber')

## 2. Find the date range from the newly created csv file.

In [22]:
spark.sql('select min(tpep_pickup_datetime) as dateRange from uber union select max(tpep_pickup_datetime) from uber').show()

+-------------------+
|          dateRange|
+-------------------+
|2016-06-01 12:30:00|
|2016-06-12 21:59:19|
+-------------------+



In [23]:
uber.select(min("tpep_pickup_datetime").alias('start_date')).show()
uber.select(max("tpep_pickup_datetime").alias('end_date')).show()

+-------------------+
|         start_date|
+-------------------+
|2016-06-01 12:30:00|
+-------------------+

+-------------------+
|           end_date|
+-------------------+
|2016-06-12 21:59:19|
+-------------------+



## 3. Aggregate data based on week to depict weekly distance of travel 

In [24]:
spark.sql("select weekofyear(tpep_pickup_datetime) as week,sum(distance_miles) as total_distance from uber group by weekofyear(tpep_pickup_datetime)").show()

+----+--------------------+
|week|      total_distance|
+----+--------------------+
|  22|3.7738647911456354E7|
|  23| 3.541426417288098E7|
+----+--------------------+



In [25]:
uber.groupBy(weekofyear("tpep_pickup_datetime").alias("date_by_week")).agg(sum("distance_miles")).orderBy("date_by_week").show()

+------------+--------------------+
|date_by_week| sum(distance_miles)|
+------------+--------------------+
|          22|3.7738647911456354E7|
|          23| 3.541426417288098E7|
+------------+--------------------+



## 4.	Identify the vendor covering the highest miles.

In [26]:
spark.sql('select VendorID from uber group by vendorID order by sum(distance_miles) desc limit 1').show()

+--------+
|VendorID|
+--------+
|       1|
+--------+



In [32]:
(uber.select("VendorID","distance_miles").groupBy('VendorID').agg(sum('distance_miles').alias('Sum_distance'))).select("VendorID").orderBy("Sum_distance",ascending=False).show(1)

+--------+
|VendorID|
+--------+
|       1|
+--------+
only showing top 1 row



## 5. Calculate the total revenue made over a period of choice

In [5]:
range_slider = widgets.IntRangeSlider(
    value=[1, 12],
    min=1, max=12., step=1,
    description='day range:',
    
)
range_slider

IntRangeSlider(value=(1, 12), description='day range:', max=12, min=1)

In [29]:
query=("select sum(total_amount) as total_revenue from uber where dayofmonth(tpep_pickup_datetime) between {} and {}"
       .format(range_slider.value[0],range_slider.value[1]))

In [30]:
spark.sql(query).show()

+-------------------+
|      total_revenue|
+-------------------+
|5.493257646035227E7|
+-------------------+



In [31]:
(uber.filter((uber["tpep_pickup_datetime"]>="2016-06-10 04:06:40") & (uber["tpep_pickup_datetime"]<="2016-06-12 16:29:14"))).agg(sum('total_amount').alias('total revenue  between 2016-06-10 04:06:40 & 2016-06-12 16:29:14')).show()

+----------------------------------------------------------------+
|total revenue  between 2016-06-10 04:06:40 & 2016-06-12 16:29:14|
+----------------------------------------------------------------+
|                                            1.3916620769971028E7|
+----------------------------------------------------------------+

